In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import wordnet
# from surprise import Reader, Dataset, SVD, evaluate
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

from uszipcode import SearchEngine
from  geopy.distance import distance


MAX_DISTANCE = 15

import warnings; warnings.simplefilter('ignore')

In [3]:
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

In [4]:

#!dir .\Data\*.tsv

In [5]:
folder = 'Data'

apps = pd.read_csv('./'+folder+'/apps.tsv', delimiter='\t',encoding='utf-8')
user_history = pd.read_csv('./'+folder+'/user_history.tsv', delimiter='\t',encoding='utf-8')
jobs = pd.read_csv('./'+folder+'/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)
users = pd.read_csv('./'+folder+'/users.tsv' ,delimiter='\t',encoding='utf-8')
test_users = pd.read_csv('./'+folder+'/test_users.tsv', delimiter='\t',encoding='utf-8')


b'Skipping line 122433: expected 11 fields, saw 12\n'
b'Skipping line 602576: expected 11 fields, saw 12\n'
b'Skipping line 990950: expected 11 fields, saw 12\n'


### EDA

In [6]:
apps.head()


,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009
2,47,1,Train,2012-04-05 02:40:27.753,2121
3,47,1,Train,2012-04-05 02:37:02.673,848187
4,47,1,Train,2012-04-05 22:44:06.653,733748


In [7]:
user_history.head()

,UserID,WindowID,Split,Sequence,JobTitle
0,47,1,Train,1,National Space Communication Programs-Special ...
1,47,1,Train,2,Detention Officer
2,47,1,Train,3,"Passenger Screener, TSA"
3,72,1,Train,1,"Lecturer, Department of Anthropology"
4,72,1,Train,2,Student Assistant


In [8]:
jobs.head()

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:11.88,2012-04-02 23:59:59


In [9]:
users.head()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0


In [10]:
test_users.head()
len(users)

389708

In [11]:
users = users.loc[users.State == 'NY']
len(users)

20590

In [12]:
users = users.sample(frac=0.2, replace=False, random_state=1)
len(users)

4118

#### Subsetting jobs in NY

In [13]:
jobs = jobs.loc[jobs.State == 'NY']
all_jobs = jobs.copy()
#jobs.to_csv("NYjobs.tsv",  sep='\t',encoding='utf-8')

#### Subsetting jobs in Zip

In [14]:
jobs.Zip5 = jobs.Zip5.fillna(0)
jobs.Zip5 = jobs.Zip5.astype(int)
jobs = jobs.loc[jobs['Zip5'] != 0]

In [15]:
len(jobs)

33679

In [16]:

jobs = jobs.sample(frac=0.2, replace=False, random_state=1)
len(jobs)

6736

In [17]:
import gc
gc.collect()

0

In [18]:

jobs.groupby(['Zip5', 'City']).size().reset_index(name='Count').sort_values('Count', ascending=False).head()

,Zip5,City,Count
3,10001,New York,630
24,10017,New York,209
414,11747,Melville,140
22,10016,New York,138
46,10036,New York,129


### Preprocessing

### Preprocessing Description and Requirements

In [19]:
import re

def preprocessor(text):
    text = text.replace('\\r', '').replace('&nbsp', '').replace('\n', '')
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [20]:

jobs['Description'] = jobs['Description'].astype(dtype='str').apply(preprocessor)

In [21]:
jobs['Requirements'] = jobs['Requirements'].astype(dtype='str').apply(preprocessor)

### Creating jobs coordinates

In [22]:
search = SearchEngine(simple_zipcode=True)

In [23]:
#lat, "long"
"""coords_1 = c
coords_2 = (34.1, -118.42)

print (round(distance(coords_1, coords_2).miles, 2))"""
search.by_zipcode("2e").lat

In [24]:

def coordinates(zipcode):
    zipcode = search.by_zipcode(zipcode)
    community = zipcode.post_office_city
    return community, "{},{}".format(zipcode.lat, zipcode.lng)


In [25]:
%%timeit
jobs["Community"] = ""
jobs["Coordinates"] = ""
for zipcode in jobs.Zip5.unique():
    community, coordinate = coordinates(zipcode)
    jobs.loc[jobs.Zip5 == zipcode, "Coordinates"] =  str(coordinate)
    jobs.loc[jobs.Zip5 == zipcode, "Community"] =  community

4.12 s ± 96.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
jobs.groupby(['Community']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Count
272,"New York, NY",2079
335,"Rochester, NY",329
47,"Buffalo, NY",241
45,"Brooklyn, NY",228
43,"Bronx, NY",165
243,"Melville, NY",141
0,"Albany, NY",109
417,"White Plains, NY",109
374,"Syracuse, NY",91
163,"Hauppauge, NY",75


In [27]:
jobs.groupby(['Community', 'Zip5', 'Coordinates']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Zip5,Coordinates,Count
386,"New York, NY",10001,"40.75,-73.99",633
399,"New York, NY",10017,"40.75,-73.973",212
398,"New York, NY",10016,"40.74,-73.97",143
354,"Melville, NY",11747,"40.79,-73.4",141
416,"New York, NY",10036,"40.76,-73.99",134
400,"New York, NY",10018,"40.76,-73.99",118
404,"New York, NY",10022,"40.76,-73.97",97
390,"New York, NY",10005,"40.705,-74.005",90
83,"Brooklyn, NY",11201,"40.7,-73.99",87
401,"New York, NY",10019,"40.77,-73.99",76


### creating user coordinates

In [28]:

def creating_coordinates(df, column):
    df["Community"] = ""
    df["Coordinates"] = "None, None"
    for zipcode in df[column].unique():
        community, coordinate = coordinates(zipcode)
        df.loc[df[column] == zipcode, "Coordinates"] =  str(coordinate)
        df.loc[df[column] == zipcode, "Community"] =  community
    return df
users = creating_coordinates(users, 'ZipCode')

In [29]:
users.groupby(['Community']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Count
49,"Brooklyn, NY",544
284,"New York, NY",446
46,"Bronx, NY",405
354,"Rochester, NY",200
397,"Staten Island, NY",90
208,"Jamaica, NY",88
458,"Yonkers, NY",62
51,"Buffalo, NY",61
138,"Flushing, NY",38
11,"Astoria, NY",36


### Distance Matrix 

In [30]:
users.head(2)

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany,Community,Coordinates
200098,248008,4,Train,New York,NY,US,10003,None,Electrical Engineering,NaN,2,15.0,Yes,No,0,"New York, NY","40.73,-73.99"
286529,1184529,5,Train,New York,NY,US,10035,High School,Not Applicable,2005-01-01 00:00:00,3,7.0,NaN,No,0,"New York, NY","40.8,-73.93"


In [31]:
jobs.head(2)

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate,Community,Coordinates
322649,268634,2,On-Call Office Services Associate (Scanning/Ma...,pitney bowes management services pbms provides...,the ideal candidate must be able to demonstrat...,Armonk,NY,US,10504,2012-04-16 14:03:48.903,2012-05-15 23:59:59,"Armonk, NY","41.13,-73.69"
1073169,936381,7,"Coordinator, Corporate Marketing",gannett corporate marketing coordinator corpo...,,New York,NY,US,10001,2012-05-24 21:05:10.253,2012-06-23 23:59:00,"New York, NY","40.75,-73.99"


In [32]:
apps.head(2)

,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009


In [33]:
user_history.head(2)

,UserID,WindowID,Split,Sequence,JobTitle
0,47,1,Train,1,National Space Communication Programs-Special ...
1,47,1,Train,2,Detention Officer


### Subset most applied jobs by title

### Looking similar jobs

In [34]:
current_app_intercep = np.intersect1d(users.UserID.unique(), apps.UserID.unique())
historical_app_intercep = np.intersect1d(users.UserID.unique(), user_history.UserID.unique())
total_users = users.UserID.unique()
with_info =  set(current_app_intercep.tolist() + historical_app_intercep.tolist())

current_app = 1-len(current_app_intercep )/len(total_users )
historical_app = 1-len(historical_app_intercep)/ len(total_users )
cold_star_p = len(with_info)/len(total_users )

print(current_app, historical_app, cold_star_p)


0.1733851384167071 0.03302574065080133 0.994900437105391


In [35]:
#distance_jobs_df = pd.DataFrame(columns=jobs.JobID.unique(), index = users.index)

### Looks for most applied jobs id

### Distance matrix

In [36]:


#distance_jobs_df = pd.DataFrame(columns=jobs.JobID.unique(), index = users.index)

In [37]:
#distance_jobs_df.to_csv()

### Recomender top k liked

In [38]:
def user_exist(user):
    if len(users.loc[users['UserID'] == user]) == 0:
        return False
    return True

def has_coordinates(user):
    if len(users.loc[users['UserID'] == user, "Coordinates"]) == 0:
        return False
    return True
    

In [39]:
unique = 0.3 #update name
top = 20

def ranking_by_popularity(top):
    
    popular_jobs = user_history.groupby(
    ['JobTitle']).size().reset_index(
    name='Count').sort_values('Count', ascending=False)
    ranking =  dict()
    top_i = 0
    
    while True:
        job_title = popular_jobs['JobTitle'].iloc[top_i]
        jobs_list = jobs.loc[jobs['Title'] == job_title, ['JobID']]['JobID'].unique().tolist()

        if len(jobs_list) > 1:
            ranking[job_title] = jobs_list

        if len(ranking) == top:
            break

        top_i +=1
    return ranking

ranking_popular = ranking_by_popularity(top)

In [40]:

def recommender_popular_jobs(user, unique, top):
    
    recommended_popular_jobs = dict()
    c1 = users.loc[users['UserID'] == user, 'Coordinates']
    
    if user_exist(user) and has_coordinates(user):
        for title, jobs_list in ranking_popular.items():
            
            distances = dict()
            
            for job in jobs_list:
                
                c2 = jobs.loc[jobs['JobID'] == job, 'Coordinates']
                if c2.iloc[0].split(',')[0] == 'None':
                    continue
                distances[job] = round(distance(c1, c2).miles, 2)
            
            distances = sorted(distances.items(), key=lambda kv: kv[1])
            closest = distances[0]
            if closest[1] >= MAX_DISTANCE:
                continue
            recommended_popular_jobs[title] = (closest[0], closest[1])
            
    else:
        for title, jobs_list in ranking_popular.items():
            recommended_popular_jobs[title] = jobs_list[0]
            
    return recommended_popular_jobs

recommender_popular_jobs(98, unique, top)

{'Customer Service Representative': 1015707,
 'Cashier': 668223,
 'Administrative Assistant': 617073,
 'Sales Associate': 407797,
 'Assistant Manager': 317522,
 'Office Manager': 203695,
 'Manager': 1097462,
 'Receptionist': 518270,
 'Customer Service': 119569,
 'Store Manager': 1075099,
 'General Manager': 286330,
 'Sales Representative': 6460,
 'Medical Assistant': 778080,
 'Project Manager': 127738,
 'Account Executive': 865392,
 'Account Manager': 158909,
 'Sales Manager': 513121,
 'Executive Assistant': 166751,
 'Sales': 1076762,
 'Teller': 376812}

### Content Based title x profile



In [41]:
jobs['profile'] = jobs['Title'].astype(str)  +  '. ' + jobs['Requirements'].astype(str) +  '. ' + jobs['Description'].astype(str) 

In [42]:
%%timeit

users['profile'] = (users['DegreeType'].astype(str) + " ")*5 + (users['Major'].astype(str) +  " " )*5
#+ " Years of Experience " + users['TotalYearsExperience'].astype(str)
#+ str(p.number_to_words(users["TotalYearsExperience"])) + " Years of Experience " + 


3.27 ms ± 25.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### Garbage words

In [43]:
users['profile'] = users['profile'].str.replace('None.', '.')
users['profile'] = users['profile'].str.replace('Not Applicable', '.')
users['profile'] = users['profile'].str.replace(' nan', ' ') 

users['profile'].iloc[1]

'High School High School High School High School High School . . . . . '

In [44]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec

import pickle

simple_preprocess(users['profile'].iloc[1])

['high',
 'school',
 'high',
 'school',
 'high',
 'school',
 'high',
 'school',
 'high',
 'school']

In [45]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))
        self.epoch += 1
        

In [46]:

users.loc[users['UserID'] == 4365]

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany,Community,Coordinates,profile


In [57]:
apps.head()

,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009
2,47,1,Train,2012-04-05 02:40:27.753,2121
3,47,1,Train,2012-04-05 02:37:02.673,848187
4,47,1,Train,2012-04-05 22:44:06.653,733748


In [126]:
def user_profile(user_id):
    user_profile = np.array(users.loc[users['UserID'] == user_id, ['DegreeType', 'Major']])[0]
    historical_apps = user_history.loc[user_history.UserID == user_id, 'JobTitle']
    user_profile = """
    Degree Type: {}
    Major: {}
    """.format(user_profile[0], user_profile[1])

    return user_profile

In [106]:
def historical_application(user_id):
    historical_apps = apps.loc[apps.UserID == user_id]['JobID']
    content = list()
    for application in historical_apps:
        temp = jobs.loc[jobs.JobID == application, ['Title', 'Description', 'Requirements']]
        if len(temp) != 0:
            content += [temp.Title.values + ". " + temp.Description.values + ". " + temp.Requirements.values]
    return content


In [96]:
%%time
def similarities_nlp_model( model_name = "jobs_doc2vec_model",
                 mapping_name = "jobID_mapping.p", max_epochs = 10,
                 alpha = 0.025):
    
    document = list()
    jobID_mapping = dict()
        
    for i, token in enumerate(jobs['profile']):
        value = jobs.iloc[i]["JobID"]
        tokens = TaggedDocument(simple_preprocess(token), [i])
        document.append(tokens)
        jobID_mapping[i] = value

    epoch_logger = EpochLogger()
    model = Doc2Vec(size = 50, alpha=alpha, 
                    min_alpha=0.00025, min_count=1,
                    callbacks=[epoch_logger], dm =1, workers=8, vector_size=5, window=2
                   )
    
    model.build_vocab(document)
    print(model.corpus_count)
    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch))
        model.train(document, 
                    total_examples=model.corpus_count, 
                    epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha

    # create a dictionary
    pickle.dump(model, open(model_name, "wb")) 
    pickle.dump(jobID_mapping, open(mapping_name, "wb")) 

# -------------------------------------------------------------
# Load the dictionary back from the pickle file.
    return model, jobID_mapping

model, jobID_mapping = similarities_nlp_model()


6736
iteration 0
Epoch #0 start
Epoch #1 start
Epoch #2 start
Epoch #3 start
Epoch #4 start
iteration 1
Epoch #5 start
Epoch #6 start
Epoch #7 start
Epoch #8 start
Epoch #9 start
iteration 2
Epoch #10 start
Epoch #11 start
Epoch #12 start
Epoch #13 start
Epoch #14 start
iteration 3
Epoch #15 start
Epoch #16 start
Epoch #17 start
Epoch #18 start
Epoch #19 start
iteration 4
Epoch #20 start
Epoch #21 start
Epoch #22 start
Epoch #23 start
Epoch #24 start
iteration 5
Epoch #25 start
Epoch #26 start
Epoch #27 start
Epoch #28 start
Epoch #29 start
iteration 6
Epoch #30 start
Epoch #31 start
Epoch #32 start
Epoch #33 start
Epoch #34 start
iteration 7
Epoch #35 start
Epoch #36 start
Epoch #37 start
Epoch #38 start
Epoch #39 start
iteration 8
Epoch #40 start
Epoch #41 start
Epoch #42 start
Epoch #43 start
Epoch #44 start
iteration 9
Epoch #45 start
Epoch #46 start
Epoch #47 start
Epoch #48 start
Epoch #49 start
Wall time: 1min 9s


In [120]:
#If user has coordinates
#if user has info then similarties if not popularity ranker
jobID_mapping = pickle.load(open("jobID_mapping.p", "rb"))
model  = pickle.load(open("jobs_doc2vec_model", "rb"))

def content_distance_based_recommender(user_id, jobID_mapping = jobID_mapping, model =  model, top = 10):
    #As infer_vector produce stochastics result I made a for to save the best list
    user_profile = np.array(users.loc[users['UserID'] == user_id, 'profile'])[0]

    historical_apps = historical_application(user_id)
    for application in historical_apps:
        user_profile += ". " + (str(application[0]) + " ")
    user_profile = simple_preprocess(user_profile)
    
    best = 0
    top10=pd.DataFrame(index = range(top), columns = ['JobID', 'Title', 'Distance', 'Description', 'Requirements'])
    c1 = users.loc[users['UserID'] == user_id, 'Coordinates']
    job_distance_list = list()
    
    
    
    for i in range (1):
        inferred_vector = model.infer_vector(user_profile)
        sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
        sum_results = 0
        count = 0
        total_recom = 0
        job_distance_list = list()
        
        while True:
            job_id = jobID_mapping[sims[count][0]]
            c2 = jobs.loc[jobs['JobID'] == jobID_mapping[sims[count][0]], 'Coordinates']
            count +=1
            if len(c2) == 0:
                #print("Empty", sims[count][0])
                continue
            
            if c2.iloc[0].split(',')[0] == 'None':
                #print('None', sims[count][0])
                continue
                
            job_distance = round(distance(c1, c2).miles, 2)
            
            if job_distance >= MAX_DISTANCE:
                #print('Distance', job_distance,  sims[count][0])
                continue

            sum_results+=sims[count][1]
            total_recom +=1
            job_distance_list.append(job_distance)
            if total_recom == top:
                break
        
        ##Best simulation
        if sum_results > best:
            best = sum_results
            best_sim = sims
            job_distance_list2 = job_distance_list
            
    for i in range(top):
        recomendation = jobID_mapping[best_sim[i][0]]
        top10.iloc[i]['JobID', 'Title',  'Description', 'Requirements'] = np.array(jobs.loc[jobs['JobID'] == recomendation][['JobID', 'Title', 
                                                               'Description', 'Requirements']])[0]
        top10.iloc[i]['Distance'] = job_distance_list2[i]
    
    return top10, sims    
 
        

In [115]:
users.head()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany,Community,Coordinates,profile
200098,248008,4,Train,New York,NY,US,10003,None,Electrical Engineering,NaN,2,15.0,Yes,No,0,"New York, NY","40.73,-73.99",.....Electrical Engineering Electrical Enginee...
286529,1184529,5,Train,New York,NY,US,10035,High School,Not Applicable,2005-01-01 00:00:00,3,7.0,NaN,No,0,"New York, NY","40.8,-73.93",High School High School High School High Schoo...
57797,1122774,1,Train,New Hartford,NY,US,13413,None,Business Administration,1987-01-01 00:00:00,2,23.0,Yes,Yes,8,"New Hartford, NY","43.06,-75.27",.....Business Administration Business Administ...
60141,1165147,1,Train,Staten Island,NY,US,10314,Bachelor's,BA/Criminal Justice,1980-06-01 00:00:00,7,35.0,Yes,Yes,200,"Staten Island, NY","40.6,-74.15",Bachelor's Bachelor's Bachelor's Bachelor's Ba...
371308,875025,7,Train,Brooklyn,NY,US,11219,None,Business Management,2009-01-01 00:00:00,8,6.0,Yes,Yes,8,"Brooklyn, NY","40.63,-74.0",.....Business Management Business Management B...


In [129]:
user_id = 248008
top_recommendations, s = content_distance_based_recommender(user_id, top=10)
print("USER PROFILE: ", user_profile(user_id))
print("HISTORICAL APPLICATION CONTENT:\n\n", historical_application(user_id))

USER PROFILE:  
    Degree Type: None
    Major: Electrical Engineering
    
HISTORICAL APPLICATION CONTENT:

 [array(['COMPLIANCE OFFICER. our client an international banking leader has vice president level situations available in its corporate compliance division for experienced banking and regulatory professionals vp compliance control room will provide guidance and advice to business lines on compliance with the bank s policies relating to transactional conflicts of interest and employee conflicts insider trading gifts entertainment outside business activities etc will also include customer firm conflicts and customer customer conflicts as well incumbent will be involved with policies procedures training evaluation and escalation of compliance control room matters will work closely with legal for interpretation and implementation and will be subject matter expert vp capital markets compliance will perform anti money laundering monitoring and testing for primarily capital markets to

In [130]:
top_recommendations

,JobID,Title,Distance,Description,Requirements
0,638497,COMPLIANCE OFFICER,1.54,our client an international banking leader has...,vp compliance control officer requires 8 to 10...
1,538665,Compliance Officer Banking$100k,2.07,classification compliance officer n ncompensat...,required skills nthe candidate should have min...
2,1072653,Compliance Coordinator,1.38,classification compliance officer n ncompensat...,in order to effectively represent the company ...
3,1043381,Capital Markets Compliance Officer - VP,1.54,our client a major international bank is curre...,5 years experience in a capital markets compl...
4,537805,Risk Analyst,1.38,classification risk analyst n ncompensation 10...,financial service organization seeks a vp in t...
5,1096860,Manager - SEC Reporting,1.9,our client is looking for a manager of sec rep...,7 to 10 years of experience public accounting...
6,643207,Underwriter II,1.38,responsibilities,
7,257448,Compliance Specialist,2.71,classification financial analyst n ncompensati...,bachelors degree in accounting finance or rela...
8,1067341,Senior Operational Risk Management Officer,2.07,senior operational risk management officer bas...,skills superior communication and organization...
9,238459,Internal Auditor,2.07,classification auditor internal n ncompensatio...,bachelors in accounting or related field and 2...


In [110]:
t

,JobID,Title,Distance,Description,Requirements
0,638497,COMPLIANCE OFFICER,1.54,our client an international banking leader has...,vp compliance control officer requires 8 to 10...
1,257448,Compliance Specialist,1.9,classification financial analyst n ncompensati...,bachelors degree in accounting finance or rela...
2,84970,Senior Compliance Associate,1.38,responsibilitiesmoody s is seeking an experien...,qualifications minimum of 5 years experience i...
3,1043381,Capital Markets Compliance Officer - VP,1.54,our client a major international bank is curre...,5 years experience in a capital markets compl...
4,1072653,Compliance Coordinator,1.38,classification compliance officer n ncompensat...,in order to effectively represent the company ...


In [107]:
t.Title

0                         COMPLIANCE OFFICER
1                      Compliance Specialist
2                Senior Compliance Associate
3    Capital Markets Compliance Officer - VP
4                     Compliance Coordinator
Name: Title, dtype: object

In [108]:
t.Description


0    our client an international banking leader has...
1    classification financial analyst n ncompensati...
2    responsibilitiesmoody s is seeking an experien...
3    our client a major international bank is curre...
4    classification compliance officer n ncompensat...
Name: Description, dtype: object

In [109]:
users

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany,Community,Coordinates,profile
200098,248008,4,Train,New York,NY,US,10003,None,Electrical Engineering,NaN,2,15.0,Yes,No,0,"New York, NY","40.73,-73.99",.....Electrical Engineering Electrical Enginee...
286529,1184529,5,Train,New York,NY,US,10035,High School,Not Applicable,2005-01-01 00:00:00,3,7.0,NaN,No,0,"New York, NY","40.8,-73.93",High School High School High School High Schoo...
57797,1122774,1,Train,New Hartford,NY,US,13413,None,Business Administration,1987-01-01 00:00:00,2,23.0,Yes,Yes,8,"New Hartford, NY","43.06,-75.27",.....Business Administration Business Administ...
60141,1165147,1,Train,Staten Island,NY,US,10314,Bachelor's,BA/Criminal Justice,1980-06-01 00:00:00,7,35.0,Yes,Yes,200,"Staten Island, NY","40.6,-74.15",Bachelor's Bachelor's Bachelor's Bachelor's Ba...
371308,875025,7,Train,Brooklyn,NY,US,11219,None,Business Management,2009-01-01 00:00:00,8,6.0,Yes,Yes,8,"Brooklyn, NY","40.63,-74.0",.....Business Management Business Management B...
325078,932674,6,Train,Levittown,NY,US,11756,Bachelor's,Business Management and Accounting,1971-01-01 00:00:00,4,26.0,NaN,No,0,"Levittown, NY","40.72,-73.52",Bachelor's Bachelor's Bachelor's Bachelor's Ba...
27089,563339,1,Train,Schenectady,NY,US,12308,None,History,1972-01-01 00:00:00,8,26.0,Yes,No,0,"Schenectady, NY","42.82,-73.92",.....History History History History History
74986,1433717,1,Train,Yorktown Heights,NY,US,10598,Bachelor's,NaN,NaN,6,9.0,No,No,0,"Yorktown Heights, NY","41.29,-73.79",Bachelor's Bachelor's Bachelor's Bachelor's Ba...
113397,944873,2,Train,Johnson City,NY,US,13790-2686,PhD,Industrial and Systems Engineering,NaN,4,11.0,Yes,No,0,None,"None,None",PhD PhD PhD PhD PhD Industrial and Systems Eng...
250123,153286,5,Train,Hollis,NY,US,11423,None,Medical/Clinical Assistant,2011-08-01 00:00:00,5,7.0,No,No,0,"Hollis, NY","40.72,-73.77",.....Medical/Clinical Assistant Medical/Clinic...


#### User to User
#### Distance Recommender



### Testing